In [76]:
import geopandas as gpd
station_id_map = gpd.read_parquet('station-id-map.parquet')

# From the station id map, we can look at the edf directly and find how long it takes to go from one stop to another
# Take Brown Line (Kimball to Kedzie-Brown to Francisco)
station_id_map[(station_id_map.line == 'T') & station_id_map.stop_name.isin(['Kimball', 'Merchandise Mart'])]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
468,30090,NaN,Merchandise Mart,None,41.888969,-87.633924,0,40460,1,Brn,T,1,POINT (-87.63392 41.88897),1271,T,31291,41.88897,-87.63397,1
469,30090,NaN,Merchandise Mart,None,41.888969,-87.633924,0,40460,1,Brn,T,1,POINT (-87.63392 41.88897),1582,T,31340,41.88897,-87.63397,1
500,30249,NaN,Kimball,None,41.967901,-87.713065,0,41290,1,Brn,T,1,POINT (-87.71307 41.96790),439,T,31735,41.96607,-87.71087,1
511,30091,NaN,Merchandise Mart,None,41.888969,-87.633924,0,40460,1,Brn,T,5,POINT (-87.63392 41.88897),968,T,35420,41.89030,-87.63548,5
512,30091,NaN,Merchandise Mart,None,41.888969,-87.633924,0,40460,1,Brn,T,5,POINT (-87.63392 41.88897),816,T,35414,41.89030,-87.63548,5
513,30091,NaN,Merchandise Mart,None,41.888969,-87.633924,0,40460,1,Brn,T,5,POINT (-87.63392 41.88897),892,T,35418,41.89030,-87.63548,5
514,30091,NaN,Merchandise Mart,None,41.888969,-87.633924,0,40460,1,Brn,T,5,POINT (-87.63392 41.88897),7781,T,35405,41.89030,-87.63548,5
540,30250,NaN,Kimball,None,41.967901,-87.713065,0,41290,1,Brn,T,5,POINT (-87.71307 41.96790),1275,T,35020,41.96790,-87.71314,5


In [77]:
#station_id_map.set_index('parent_station').to_dict()['CurrentStationId']
from collections import defaultdict
PS_to_CS_dict = defaultdict(list)
for idx,row in station_id_map.iterrows():
    PS_to_CS_dict[row['parent_station']].append(row['CurrentStationId'])

print(PS_to_CS_dict)

defaultdict(<class 'list'>, {40010: [11080, 15640], 40060: [11520, 15200], 40070: [11300, 15420], 40180: [11060, 15660], 40220: [11160, 15560], 40230: [11631, 11622, 11633, 11640, 15067, 15062, 15065, 15080, 15076, 15078, 15074], 40250: [11140, 15578], 40320: [11420, 15300, 15281], 40350: [11235, 11232, 11230, 11240, 11228, 15480], 40370: [11340, 15380], 40390: [11020, 15695], 40430: [11260, 15460], 40470: [11206, 11203, 11202, 11220, 11201, 15500], 40490: [11380, 15340], 40550: [11550, 11560, 15160], 40570: [11480, 15240], 40590: [11440, 15280], 40670: [11460, 15260], 40750: [11620, 11604, 11605, 15100, 15083, 15082, 15095, 15097, 15093, 15087, 15085], 40790: [11320, 15400], 40810: [11180, 15540], 40820: [11642, 11646, 11653, 11643, 11651, 11660, 15041, 15026, 15047, 15023, 15057, 15062, 15043, 15045, 15055, 15060], 40890: [11663, 11662, 11699, 11667, 11693, 11696, 11673, 11682, 11675, 15020], 40920: [11120, 15600], 40970: [11100, 15620], 40980: [11040, 15680], 41020: [11500, 15231, 1

In [78]:
# In previous result, scroll over to see CurrentStationId -> 31735, 31340
#
# Now, let's look at the actual times from the entries data
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')
edf.query('RunNumber == 420 and Datetime > "2022-03-01 00:53:00"').head()

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
11202,T,2022-03-01 00:53:02,41.94702,-87.67474,420,31580,31740,1.620330,Brn,to,Kimball,False,ttpm_predslip_brown,
11268,T,2022-03-01 00:53:22,41.94702,-87.67474,420,31580,31740,1.620330,Brn,to,Kimball,False,ttpm_predslip_brown,
11333,T,2022-03-01 00:53:42,41.95077,-87.67483,420,31581,31740,1.600769,Brn,to,Kimball,False,ttpm_predslip_brown,
11398,T,2022-03-01 00:54:01,41.95077,-87.67483,420,31581,31740,1.600769,Brn,to,Kimball,False,ttpm_predslip_brown,
11462,T,2022-03-01 00:54:21,41.95451,-87.67496,420,31600,31740,1.650667,Brn,to,Kimball,False,ttpm_predslip_brown,


In [79]:
# Now, look at a RunNumber on the Brown Line and just those stations in the "1" direction
CurrentStationIdOne = 31735 #Kimball
CurrentStationIdTwo = 31340 #Merchandise Mart
edf.query(f'RunNumber == 420 and CurrentStationId.isin([{CurrentStationIdOne}, {CurrentStationIdTwo}])').head()

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
6719,T,2022-03-01 00:30:42,41.88897,-87.63397,420,31340,31740,1.614542,Brn,to,Kimball,False,ttpm_predslip_brown,
6786,T,2022-03-01 00:31:01,41.88897,-87.63397,420,31340,31740,1.614542,Brn,to,Kimball,False,ttpm_predslip_brown,
13119,T,2022-03-01 01:03:01,41.96607,-87.71087,420,31735,31740,3.142090,Brn,to,Kimball,False,ttpm_predslip_brown,
108404,T,2022-03-01 14:04:42,41.88897,-87.63397,420,31340,31740,1.614542,Brn,to,Kimball,False,ttpm_predslip_brown,
108491,T,2022-03-01 14:05:01,41.88897,-87.63397,420,31340,31740,1.614542,Brn,to,Kimball,False,ttpm_predslip_brown,


In [80]:
# We have multiple entries for some stations (the train might stay there for a minute)
two_stations = edf.query(f'RunNumber == 420 and CurrentStationId.isin([{CurrentStationIdOne}, {CurrentStationIdTwo}])')
two_stations_dedup = two_stations[two_stations.CurrentStationId.diff() != 0]

# compute the difference
two_stations_dedup.Datetime.diff()

6719                 NaT
13119    0 days 00:32:19
108404   0 days 13:01:41
139655   0 days 02:03:19
199112   0 days 05:01:45
206005   0 days 00:31:15
221925   0 days 01:05:20
255332   0 days 02:11:21
355267   0 days 12:54:00
364335   0 days 00:35:00
380795   0 days 01:02:39
389838   0 days 00:39:00
36591    0 days 21:44:00
71623    0 days 02:12:40
136986   0 days 05:10:41
168844   0 days 02:07:40
184670   0 days 01:01:20
192902   0 days 00:37:39
290590   0 days 12:58:20
322428   0 days 02:08:20
384178   0 days 05:08:20
391488   0 days 00:34:20
406431   0 days 01:01:41
77525    1 days 01:02:39
98403    0 days 02:59:40
113392   0 days 02:32:20
232876   0 days 20:15:40
248358   0 days 02:16:01
258858   0 days 01:31:59
262256   0 days 00:32:01
268368   0 days 01:08:59
270556   0 days 00:30:20
Name: Datetime, dtype: timedelta64[ns]

In [81]:
# the actual times are betweeen 31735 and 31720 so we want every other one, starting with 2
two_stations_dedup.Datetime.diff().iloc[1::2] # 2 is the start, empty is the end and 2 is the step size

13119    0 days 00:32:19
139655   0 days 02:03:19
206005   0 days 00:31:15
255332   0 days 02:11:21
364335   0 days 00:35:00
389838   0 days 00:39:00
71623    0 days 02:12:40
168844   0 days 02:07:40
192902   0 days 00:37:39
322428   0 days 02:08:20
391488   0 days 00:34:20
77525    1 days 01:02:39
113392   0 days 02:32:20
248358   0 days 02:16:01
262256   0 days 00:32:01
270556   0 days 00:30:20
Name: Datetime, dtype: timedelta64[ns]

In [82]:
import pandas as pd
PS_to_CS_df = pd.merge(station_id_map[['parent_station', 'CurrentStationId', 'stop_name']], edf[['CurrentStationId', 'Datetime', 'Line', 'RunNumber']], on='CurrentStationId', how='left')
#print(PS_to_CS_df.to_string())
run420 = PS_to_CS_df.query('Line == "T" and Datetime < "2022-03-02 12:00:00" and RunNumber == 420').sort_values('Datetime') #.drop_duplicates('CurrentStationId').to_string())
run420

,parent_station,CurrentStationId,stop_name,Datetime,Line,RunNumber
745427,41310,35200,Paulina,2022-03-01 00:00:01,T,420
745428,41310,35200,Paulina,2022-03-01 00:00:21,T,420
705535,40360,35220,Southport,2022-03-01 00:01:02,T,420
705536,40360,35220,Southport,2022-03-01 00:01:22,T,420
747181,41320,35280,Belmont,2022-03-01 00:04:01,T,420
...,...,...,...,...,...,...
679355,40870,31700,Francisco,2022-03-02 00:56:01,T,420
682035,41180,31720,Kedzie-Brown,2022-03-02 00:57:01,T,420
682036,41180,31720,Kedzie-Brown,2022-03-02 00:57:22,T,420
687402,41290,31735,Kimball,2022-03-02 00:57:42,T,420


In [83]:
print(run420[run420.parent_station.shift()!=run420.parent_station].to_string())

# compare runs (take statistics across all runs?)
# does time of day take longer?

        parent_station  CurrentStationId                  stop_name            Datetime Line  RunNumber
745427           41310             35200                    Paulina 2022-03-01 00:00:01    T        420
705535           40360             35220                  Southport 2022-03-01 00:01:02    T        420
747181           41320             35280                    Belmont 2022-03-01 00:04:01    T        420
741556           41210             35300                 Wellington 2022-03-01 00:05:01    T        420
715318           40530             35320                   Diversey 2022-03-01 00:06:02    T        420
742176           41220             35340                  Fullerton 2022-03-01 00:07:42    T        420
716326           40660             35360                   Armitage 2022-03-01 00:09:01    T        420
402263           40800             35363                   Sedgwick 2022-03-01 00:09:41    T        420
391939           40710             35382              Chicago-Br

In [84]:
run420[run420.parent_station.shift()!=run420.parent_station].query('parent_station == (40460, 41180) and CurrentStationId > 35000').Datetime.diff().iloc[2::2]
# get differences and filter out initial (long break) and times it took to go back

379390   0 days 00:34:21
714831   0 days 00:29:20
372940   0 days 00:29:40
372954   0 days 00:24:20
372964   0 days 00:29:21
Name: Datetime, dtype: timedelta64[ns]

In [85]:
run420[run420.parent_station.shift()!=run420.parent_station].query('parent_station == (40460, 41180) and CurrentStationId < 35000').Datetime.diff().iloc[1::2]
#check the same stations as above, but going in the opposite direction, same run

681705   0 days 00:32:20
681807   0 days 00:32:39
681864   0 days 00:32:40
681960   0 days 00:31:20
681988   0 days 00:34:00
682035   0 days 00:35:40
Name: Datetime, dtype: timedelta64[ns]

In [86]:
run428= PS_to_CS_df.query('Line == "T" and Datetime < "2022-03-02 12:00:00" and RunNumber == 428').sort_values('Datetime')
print(run428[run428.parent_station.shift()!=run428.parent_station].to_string())

        parent_station  CurrentStationId                  stop_name            Datetime Line  RunNumber
666478           40710             31345              Chicago-Brown 2022-03-01 00:00:01    T        428
321213           40800             31363                   Sedgwick 2022-03-01 00:06:42    T        428
651693           40660             31382                   Armitage 2022-03-01 00:10:01    T        428
339968           41220             31420                  Fullerton 2022-03-01 00:15:21    T        428
650341           40530             31438                   Diversey 2022-03-01 00:17:22    T        428
640144           40360             31540                  Southport 2022-03-01 00:23:21    T        428
687956           41310             31560                    Paulina 2022-03-01 00:24:41    T        428
691742           41440             31580              Addison-Brown 2022-03-01 00:26:01    T        428
694553           41460             31600          Irving Park-Br

In [87]:
run428[run428.parent_station.shift()!=run428.parent_station].query('parent_station == (40460, 41180) and CurrentStationId > 35000').Datetime.diff().iloc[1::]

714800   0 days 00:28:21
739090   0 days 01:11:39
711772   0 days 00:30:40
739117   0 days 01:13:21
379338   0 days 00:31:20
739365   0 days 16:21:19
379513   0 days 00:31:20
714876   0 days 01:37:00
739455   0 days 01:36:40
712032   0 days 00:33:20
739483   0 days 01:05:00
714888   0 days 00:30:00
739512   0 days 01:09:40
712062   0 days 00:30:40
Name: Datetime, dtype: timedelta64[ns]

In [88]:
### Predictions

In [89]:
import pandas as pd
pdf = pd.read_parquet('2022-03-preds-start.parquet')
print(pdf.query('RunNumber == 420 and Datetime < "2022-03-01 12:00:00"').to_string())

      Line            Datetime  CurrentStationId  RunNumber  StopOrder  ParentStop                                   StopName Prediction
436      T 2022-03-01 00:00:01             35200        420          0       40360                                  Southport        Due
437      T 2022-03-01 00:00:01             35200        420          1       41320                                    Belmont          4
438      T 2022-03-01 00:00:01             35200        420          2       41210                                 Wellington          5
439      T 2022-03-01 00:00:01             35200        420          3       40530                                   Diversey          6
440      T 2022-03-01 00:00:01             35200        420          4       41220                                  Fullerton          8
441      T 2022-03-01 00:00:01             35200        420          5       40660                                   Armitage         10
442      T 2022-03-01 00:00:01           

In [90]:
# want a map from parent_station (aka ParentStop in the pdf) to CurrentStationId for Brown Line
# station_id_map.CurrentStationId // 1000 == 31 find the stations on the brown line (3) going in direction (1) (Kimball)
brown_entries = station_id_map[(station_id_map.line == 'T') & (station_id_map.CurrentStationId // 1000 == 31)].sort_values('CurrentStationId')[['stop_name', 'parent_station','CurrentStationId']]

In [91]:
brown_entries.query('stop_name == "Merchandise Mart"')

,stop_name,parent_station,CurrentStationId
468,Merchandise Mart,40460,31291
469,Merchandise Mart,40460,31340


In [92]:
bmap = brown_entries[['parent_station','CurrentStationId']].set_index('parent_station').CurrentStationId.to_dict()

In [93]:
# Get run 420 again
run = pdf.query("RunNumber == 420")

# map the ParentStop through the bmap to get the StationId for the Prediction
run_with_id = run.assign(PredictionStationId=run.ParentStop.map(bmap))

In [94]:
run_with_id[(run_with_id.CurrentStationId == CurrentStationIdOne) & (run_with_id.PredictionStationId == CurrentStationIdTwo)]

,Line,Datetime,CurrentStationId,RunNumber,StopOrder,ParentStop,StopName,Prediction,PredictionStationId


In [95]:
'''pd.merge(PS_to_CS_df, pdf[['Prediction']], on=['RunNumber','Datetime'], how='left')'''
#merge on run num and datetime
#how do these match up? How can we present the predictions? 
#Given a run num and a window of time, look at predictions, how does predictions change over time

"pd.merge(PS_to_CS_df, pdf[['Prediction']], on=['RunNumber','Datetime'], how='left')"

In [96]:
### Data 

In [97]:
import geopandas as gpd
station_id_map = gpd.read_parquet('station-id-map.parquet')

## How long between stops?

In [98]:
# From the station id map, we can look at the edf directly and find how long it takes to go from one stop to another
# Take Brown Line (Irving Park-Brown to Damen-Brown)
station_id_map[(station_id_map.line == 'T') & station_id_map.stop_name.isin(['Irving Park-Brown','Damen-Brown'])]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
463,30018,NaN,Damen-Brown,None,41.966286,-87.678639,0,40090,1,Brn,T,1,POINT (-87.67864 41.96629),365,T,31640,41.96641,-87.67864,1
505,30281,NaN,Irving Park-Brown,None,41.954521,-87.674868,0,41460,1,Brn,T,1,POINT (-87.67487 41.95452),1273,T,31600,41.95451,-87.67496,1
509,30019,NaN,Damen-Brown,None,41.966286,-87.678639,0,40090,1,Brn,T,5,POINT (-87.67864 41.96629),62,T,35120,41.96641,-87.67864,5
544,30282,NaN,Irving Park-Brown,None,41.954521,-87.674868,0,41460,1,Brn,T,5,POINT (-87.67487 41.95452),737,T,35160,41.95451,-87.67496,5


In [99]:
# In previous result, scroll over to see CurrentStationId -> 31600, 31640
#
# Now, let's look at the actual times from the entries data
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')

In [100]:
edf.query('RunNumber == 420 and Datetime > "2022-03-01 00:53:00"')

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
11202,T,2022-03-01 00:53:02,41.94702,-87.67474,420,31580,31740,1.620330,Brn,to,Kimball,False,ttpm_predslip_brown,
11268,T,2022-03-01 00:53:22,41.94702,-87.67474,420,31580,31740,1.620330,Brn,to,Kimball,False,ttpm_predslip_brown,
11333,T,2022-03-01 00:53:42,41.95077,-87.67483,420,31581,31740,1.600769,Brn,to,Kimball,False,ttpm_predslip_brown,
11398,T,2022-03-01 00:54:01,41.95077,-87.67483,420,31581,31740,1.600769,Brn,to,Kimball,False,ttpm_predslip_brown,
11462,T,2022-03-01 00:54:21,41.95451,-87.67496,420,31600,31740,1.650667,Brn,to,Kimball,False,ttpm_predslip_brown,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270492,T,2022-03-07 07:42:01,41.96613,-87.70430,420,31708,31740,3.148721,Brn,to,Kimball,False,ttpm_predslip_brown,
270514,T,2022-03-07 07:42:21,41.96612,-87.70583,420,31715,31740,3.148721,Brn,to,Kimball,False,ttpm_predslip_brown,
270535,T,2022-03-07 07:42:41,41.96609,-87.70882,420,31720,31740,3.154367,Brn,to,Kimball,False,ttpm_predslip_brown,
270556,T,2022-03-07 07:43:01,41.96607,-87.71087,420,31735,31740,3.142090,Brn,to,Kimball,False,ttpm_predslip_brown,


In [101]:
# Now, look at a RunNumber on the Brown Line and just those stations in the "1" direction
edf.query('RunNumber == 420 and CurrentStationId.isin([31600, 31640])') 

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
11462,T,2022-03-01 00:54:21,41.95451,-87.67496,420,31600,31740,1.650667,Brn,to,Kimball,False,ttpm_predslip_brown,
11525,T,2022-03-01 00:54:41,41.95451,-87.67496,420,31600,31740,1.650667,Brn,to,Kimball,False,ttpm_predslip_brown,
11971,T,2022-03-01 00:57:01,41.96641,-87.67864,420,31640,31740,3.158250,Brn,to,Kimball,False,ttpm_predslip_brown,
12035,T,2022-03-01 00:57:21,41.96641,-87.67864,420,31640,31740,3.158250,Brn,to,Kimball,False,ttpm_predslip_brown,
12099,T,2022-03-01 00:57:41,41.96641,-87.67864,420,31640,31740,3.158250,Brn,to,Kimball,False,ttpm_predslip_brown,
114921,T,2022-03-01 14:28:42,41.95451,-87.67496,420,31600,31740,1.650667,Brn,to,Kimball,False,ttpm_predslip_brown,
115009,T,2022-03-01 14:29:01,41.95451,-87.67496,420,31600,31740,1.650667,Brn,to,Kimball,False,ttpm_predslip_brown,
115627,T,2022-03-01 14:31:21,41.96641,-87.67864,420,31640,31740,3.158250,Brn,to,Kimball,False,ttpm_predslip_brown,
115714,T,2022-03-01 14:31:41,41.96641,-87.67864,420,31640,31740,3.158250,Brn,to,Kimball,False,ttpm_predslip_brown,
115801,T,2022-03-01 14:32:01,41.96641,-87.67864,420,31640,31740,3.158250,Brn,to,Kimball,False,ttpm_predslip_brown,


In [102]:
# We have multiple entries for some stations (the train might stay there for a minute)
two_stations = edf.query('RunNumber == 420 and CurrentStationId.isin([31640, 31600])')
two_stations_dedup = two_stations[two_stations.CurrentStationId.diff() != 0]

In [103]:
# compute the difference
two_stations_dedup.Datetime.diff()

11462                NaT
11971    0 days 00:02:40
114921   0 days 13:31:41
115627   0 days 00:02:39
137739   0 days 01:28:20
138429   0 days 00:03:01
204182   0 days 05:30:19
204774   0 days 00:02:40
228797   0 days 01:36:01
229533   0 days 00:02:39
253446   0 days 01:34:20
254103   0 days 00:03:00
361950   0 days 13:26:00
362743   0 days 00:03:00
387406   0 days 01:37:00
388237   0 days 00:03:41
43070    0 days 22:14:39
43890    0 days 00:03:00
68965    0 days 01:34:41
69906    0 days 00:03:59
142115   0 days 05:41:00
142769   0 days 00:03:00
166096   0 days 01:32:01
167035   0 days 00:03:19
189708   0 days 01:30:40
190437   0 days 00:03:20
299136   0 days 13:42:00
299941   0 days 00:03:00
320091   0 days 01:22:20
320892   0 days 00:03:40
389173   0 days 05:39:00
389918   0 days 00:03:20
412858   0 days 01:33:00
413652   0 days 00:03:00
436903   0 days 01:36:00
437593   0 days 00:03:20
61978    0 days 21:05:40
62491    0 days 00:03:20
75923    0 days 01:35:01
76285    0 days 00:02:59


In [104]:
# the actual times are betweeen 31640 and 31600 so we want every other one, starting with 1
two_stations_dedup.Datetime.diff().iloc[1::2] # 1 is the start, empty is the end and 2 is the step size

11971    0 days 00:02:40
115627   0 days 00:02:39
138429   0 days 00:03:01
204774   0 days 00:02:40
229533   0 days 00:02:39
254103   0 days 00:03:00
362743   0 days 00:03:00
388237   0 days 00:03:41
43890    0 days 00:03:00
69906    0 days 00:03:59
142769   0 days 00:03:00
167035   0 days 00:03:19
190437   0 days 00:03:20
299941   0 days 00:03:00
320892   0 days 00:03:40
389918   0 days 00:03:20
413652   0 days 00:03:00
437593   0 days 00:03:20
62491    0 days 00:03:20
76285    0 days 00:02:59
101564   0 days 00:03:40
112640   0 days 00:03:20
235700   0 days 00:02:39
247765   0 days 00:02:39
261643   0 days 00:03:00
270228   0 days 00:03:00
Name: Datetime, dtype: timedelta64[ns]

In [105]:
# we can see that it's usually around 3 minutes to go between these two stops

## What about Predictions?

Need to map the predicted stop information to the edf CurrentStationIds.

In [106]:
import pandas as pd
pdf = pd.read_parquet('2022-03-preds-start.parquet')

In [107]:
# want a map from parent_station (aka ParentStop in the pdf) to CurrentStationId for Brown Line
# station_id_map.CurrentStationId // 1000 == 31 find the stations on the brown line (3) going in direction (1) (Kimball)
brown_entries = station_id_map[(station_id_map.line == 'T') & (station_id_map.CurrentStationId // 1000 == 31)].sort_values('CurrentStationId')[['stop_name', 'parent_station','CurrentStationId']]

In [108]:
brown_entries.query('stop_name == "Belmont"')

,stop_name,parent_station,CurrentStationId
502,Belmont,41320,31475


In [109]:
bmap = brown_entries[['parent_station','CurrentStationId']].set_index('parent_station').CurrentStationId.to_dict()

In [110]:
# Get run 420 again
run = pdf.query("RunNumber == 420")

In [111]:
# map the ParentStop through the bmap to get the StationId for the Prediction
run_with_id = run.assign(PredictionStationId=run.ParentStop.map(bmap))

In [112]:
# look at the normal prediction for 31600 to 31060
run_with_id[(run_with_id.CurrentStationId == 31600) & (run_with_id.PredictionStationId == 31640)]

,Line,Datetime,CurrentStationId,RunNumber,StopOrder,ParentStop,StopName,Prediction,PredictionStationId
54281,T,2022-03-01 00:54:21,31600,420,1,40090,Damen,3,31640
546782,T,2022-03-01 14:28:42,31600,420,1,40090,Damen,3,31640
1649227,T,2022-03-02 14:17:41,31600,420,1,40090,Damen,2,31640
1757082,T,2022-03-02 15:57:41,31600,420,1,40090,Damen,3,31640
199675,T,2022-03-03 14:16:01,31600,420,1,40090,Damen,3,31640
300413,T,2022-03-03 15:53:42,31600,420,1,40090,Damen,2,31640
610576,T,2022-03-03 21:38:41,31600,420,1,40090,Damen,3,31640
814181,T,2022-03-04 00:47:41,31600,420,1,40090,Damen,3,31640
1292269,T,2022-03-04 14:33:01,31600,420,1,40090,Damen,3,31640
1781404,T,2022-03-04 23:17:21,31600,420,1,40090,Damen,3,31640


In [113]:
# we can see that the prediction is 2 or 3 minutes which lines up with what we see!

### YELLOW LINE

In [152]:
import geopandas as gpd
station_id_map = gpd.read_parquet('station-id-map.parquet').sort_values("CurrentStationId", ascending=False)


In [153]:

station_id_map.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
640,30027,NaN,Dempster-Skokie,None,42.038951,-87.751919,0,40140,1,Y,Y,5,POINT (-87.75192 42.03895),2786,Y,95080,0.0,0.0,5
641,30176,NaN,Howard,None,42.019063,-87.672892,0,40900,1,Y,Y,5,POINT (-87.67289 42.01906),2786,Y,95080,0.0,0.0,5
642,30298,NaN,Oakton-Skokie,None,42.026243,-87.747221,0,41680,1,Y,Y,5,POINT (-87.74722 42.02624),2786,Y,95080,0.0,0.0,5
637,30027,NaN,Dempster-Skokie,None,42.038951,-87.751919,0,40140,1,Y,Y,5,POINT (-87.75192 42.03895),2641,Y,95078,0.0,0.0,5
638,30176,NaN,Howard,None,42.019063,-87.672892,0,40900,1,Y,Y,5,POINT (-87.67289 42.01906),2641,Y,95078,0.0,0.0,5


In [ ]:
pd.set_option('display.max_rows', 500)
import geopandas as gpd
station_id_map = gpd.read_parquet('station-id-map.parquet')
# From the station id map, we can look at the edf directly and find how long it takes to go from one stop to another
# Take Yellow Line (Howard to Oakton-Skokie to Dempster-Skokie)
station_id_map[(station_id_map.line == 'Y') & station_id_map.stop_name.isin(['Oakton-Skokie','Dempster-Skokie' ])]

In [ ]:
pd.set_option("display.max_rows", 500)
# In previous result, scroll over to see CurrentStationId -> 31735, 31340
#
# Now, let's look at the actual times from the entries data
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')
edf.query('RunNumber == 594 and Datetime > "2022-03-01 00:53:00"').head(100)

In [138]:
# Now, look at a RunNumber on the Brown Line and just those stations in the "1" direction
YellowDempsterSkokie = 91078 #Dempster-Skokie
YellowOaktonSkokie = 91058 #Oakton-Skokie
'''CurrentStationIdThree = 91020 #Howard'''
edf.query(f'RunNumber == 594 and CurrentStationId.isin([{YellowDempsterSkokie}, {YellowOaktonSkokie}])').head()

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
2211,Y,2022-03-01 00:09:41,42.02686,-87.74745,594,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
2283,Y,2022-03-01 00:10:01,42.02686,-87.74745,594,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
2935,Y,2022-03-01 00:13:01,42.03895,-87.75192,594,91078,95090,5.069098,Yel,to,Skokie,False,ttpm_predslip_yellow,
8635,Y,2022-03-01 00:40:01,42.02686,-87.74745,594,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,
106338,Y,2022-03-01 13:57:01,42.02686,-87.74745,594,91058,95090,4.993097,Yel,to,Skokie,False,ttpm_predslip_yellow,


In [142]:
# We have multiple entries for some stations (the train might stay there for a minute)
YellowDempsterToYellowOaktonSkokie = edf.query(f'RunNumber == 594 and CurrentStationId.isin([{YellowDempsterSkokie}, {YellowOaktonSkokie }])')
YellowDempsterToYellowOaktonSkokie = YellowDempsterToYellowOaktonSkokie[YellowDempsterToYellowOaktonSkokie.CurrentStationId.diff() != 0]

# compute the difference
YellowDempsterToYellowOaktonSkokie.Datetime.diff()

2211                 NaT
2935     0 days 00:03:20
8635     0 days 00:27:00
114250   0 days 13:46:21
121016   0 days 00:25:19
121708   0 days 00:02:40
128188   0 days 00:25:20
222066   0 days 07:27:20
229031   0 days 00:25:41
229675   0 days 00:02:19
236422   0 days 00:25:20
237029   0 days 00:02:21
243924   0 days 00:26:20
252525   0 days 00:35:39
258032   0 days 00:26:41
258489   0 days 00:02:19
355474   0 days 12:39:01
356215   0 days 00:02:59
363131   0 days 00:26:41
364024   0 days 00:03:19
371225   0 days 00:27:01
372024   0 days 00:03:00
378557   0 days 00:24:59
379225   0 days 00:02:40
385869   0 days 00:27:40
386537   0 days 00:03:00
393096   0 days 00:30:00
393704   0 days 00:03:00
472634   0 days 06:14:00
473244   0 days 00:02:20
480472   0 days 00:27:00
481299   0 days 00:03:00
488969   0 days 00:28:21
489865   0 days 00:03:19
497123   0 days 00:27:40
497850   0 days 00:03:00
504933   0 days 00:29:40
505614   0 days 00:03:00
511467   0 days 00:27:00
512033   0 days 00:02:40


In [143]:
# the actual times are betweeen 31735 and 31720 so we want every other one, starting with 2
YellowDempsterToYellowOaktonSkokie.Datetime.diff().iloc[1::2] # 2 is the start, empty is the end and 2 is the step size

2935     0 days 00:03:20
114250   0 days 13:46:21
121708   0 days 00:02:40
222066   0 days 07:27:20
229675   0 days 00:02:19
237029   0 days 00:02:21
252525   0 days 00:35:39
258489   0 days 00:02:19
356215   0 days 00:02:59
364024   0 days 00:03:19
372024   0 days 00:03:00
379225   0 days 00:02:40
386537   0 days 00:03:00
393704   0 days 00:03:00
473244   0 days 00:02:20
481299   0 days 00:03:00
489865   0 days 00:03:19
497850   0 days 00:03:00
505614   0 days 00:03:00
512033   0 days 00:02:40
37096    0 days 00:03:00
45878    0 days 00:02:40
53804    0 days 00:02:59
61834    0 days 00:02:59
150013   0 days 06:16:19
157854   0 days 00:03:00
166053   0 days 00:03:00
174203   0 days 00:03:00
182484   0 days 00:02:59
296936   0 days 13:39:00
304363   0 days 00:03:00
312433   0 days 00:02:59
326099   0 days 00:30:20
331959   0 days 00:02:39
403635   0 days 05:14:21
411664   0 days 00:03:39
427764   0 days 00:36:01
434416   0 days 00:02:41
440556   0 days 00:03:00
55968    0 days 00:03:00


In [121]:
import pandas as pd
pdf = pd.read_parquet('2022-03-preds-start.parquet')

In [154]:
yellow_entries = station_id_map[(station_id_map.line == 'Y') & (station_id_map.CurrentStationId // 1000 == 95)].sort_values('CurrentStationId')[['stop_name', 'parent_station','CurrentStationId']]

In [144]:
yellow_entries.query('stop_name == "Dempster-Skokie"')

,stop_name,parent_station,CurrentStationId
649,Dempster-Skokie,40140,95021
658,Dempster-Skokie,40140,95030
625,Dempster-Skokie,40140,95035
601,Dempster-Skokie,40140,95040
652,Dempster-Skokie,40140,95041
655,Dempster-Skokie,40140,95042
604,Dempster-Skokie,40140,95045
607,Dempster-Skokie,40140,95048
643,Dempster-Skokie,40140,95052
610,Dempster-Skokie,40140,95058


In [124]:
yellow_entries.query('stop_name == "Oakton-Skokie"')

,stop_name,parent_station,CurrentStationId
651,Oakton-Skokie,41680,95021
660,Oakton-Skokie,41680,95030
627,Oakton-Skokie,41680,95035
603,Oakton-Skokie,41680,95040
654,Oakton-Skokie,41680,95041
657,Oakton-Skokie,41680,95042
606,Oakton-Skokie,41680,95045
609,Oakton-Skokie,41680,95048
645,Oakton-Skokie,41680,95052
612,Oakton-Skokie,41680,95058


In [125]:
yellow_entries.query('stop_name == "Howard"')

,stop_name,parent_station,CurrentStationId
650,Howard,40900,95021
659,Howard,40900,95030
626,Howard,40900,95035
602,Howard,40900,95040
653,Howard,40900,95041
656,Howard,40900,95042
605,Howard,40900,95045
608,Howard,40900,95048
644,Howard,40900,95052
611,Howard,40900,95058


In [155]:
ymap = yellow_entries[['parent_station','CurrentStationId']].set_index('parent_station').CurrentStationId.to_dict()

In [156]:
# Get run 120 again
run = pdf.query("RunNumber == 594")

### BLUE LINE

In [127]:
import geopandas as gpd
station_id_map = gpd.read_parquet('station-id-map.parquet')

# From the station id map, we can look at the edf directly and find how long it takes to go from one stop to another
# Take Blue Line (Forest Park to Harlem-Forest Park)
station_id_map[(station_id_map.line == 'B') & station_id_map.stop_name.isin(['Forest Park', 'Harlem-Forest Park'])]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
17,30076,NaN,Forest Park,None,41.874257,-87.817318,0,40390,1,Blue,B,1,POINT (-87.81732 41.87426),31,B,11020,41.87426,-87.81732,1
52,30189,NaN,Harlem-Forest Park,None,41.873490,-87.806961,0,40980,0,Blue,B,1,POINT (-87.80696 41.87349),476,B,11040,41.87349,-87.80696,1
81,30077,NaN,Forest Park,None,41.874257,-87.817318,0,40390,1,Blue,B,5,POINT (-87.81732 41.87426),1941,B,15695,41.87376,-87.80963,5
112,30190,NaN,Harlem-Forest Park,None,41.873490,-87.806961,0,40980,0,Blue,B,5,POINT (-87.80696 41.87349),1634,B,15680,41.87349,-87.80696,5


In [128]:
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')
edf.query('RunNumber == 120')

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
29,B,2022-03-01 00:00:01,41.97208,-87.76551,120,11602,11700,2.743046,Blu,to,O'Hare,False,ttpm_predslip_blue,
103,B,2022-03-01 00:00:21,41.98232,-87.80890,120,11604,11700,3.151556,Blu,to,O'Hare,False,ttpm_predslip_blue,
176,B,2022-03-01 00:00:41,41.97496,-87.77076,120,11607,11700,2.623162,Blu,to,O'Hare,False,ttpm_predslip_blue,
250,B,2022-03-01 00:01:02,41.97580,-87.77222,120,11608,11700,2.622869,Blu,to,O'Hare,False,ttpm_predslip_blue,
324,B,2022-03-01 00:01:22,41.97800,-87.77609,120,11610,11700,2.623537,Blu,to,O'Hare,False,ttpm_predslip_blue,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91155,B,2022-03-06 01:49:21,41.98384,-87.84917,120,11650,11660,3.178393,Blu,to,Rosemont,False,ttpm_predslip_blue,
91195,B,2022-03-06 01:49:41,41.98351,-87.85939,120,11651,11660,3.176772,Blu,to,Rosemont,False,ttpm_predslip_blue,
91235,B,2022-03-06 01:50:01,41.98366,-87.85574,120,11655,11660,3.181652,Blu,to,Rosemont,False,ttpm_predslip_blue,
91275,B,2022-03-06 01:50:21,41.98363,-87.85659,120,11658,11660,3.181652,Blu,to,Rosemont,False,ttpm_predslip_blue,


In [129]:
# In previous result, scroll over to see CurrentStationId -> 11020, 11040
#
# Now, let's look at the actual times from the entries data
import pandas as pd
edf = pd.read_parquet('2022-03-entries-start.parquet')
edf.query('RunNumber == 120 and Datetime > "2022-03-01 00:53:00"').head()

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
139466,B,2022-03-01 16:07:21,41.87426,-87.81732,120,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
139544,B,2022-03-01 16:07:41,41.87426,-87.81732,120,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
139620,B,2022-03-01 16:08:01,41.87376,-87.80970,120,11035,11700,6.182494,Blu,to,O'Hare,False,ttpm_predslip_blue,
139694,B,2022-03-01 16:08:22,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,
139767,B,2022-03-01 16:08:42,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,


In [130]:
# Now, look at a RunNumber on the Brown Line and just those stations in the "1" direction
BlueForestPark =  11020 #BlueForestPark
BlueHarlemForestPark = 11040 #BlueHarlem-ForestPark
edf.query(f'RunNumber == 120 and CurrentStationId.isin([{BlueForestPark}, {BlueHarlemForestPark}])').head()

,Line,Datetime,Lat,Lng,RunNumber,CurrentStationId,ExitStationId,Direction,LineName,DirMod,DestName,IsSched,CSSClass,Flags
139466,B,2022-03-01 16:07:21,41.87426,-87.81732,120,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
139544,B,2022-03-01 16:07:41,41.87426,-87.81732,120,11020,11700,6.200427,Blu,to,O'Hare,False,ttpm_predslip_blue,
139694,B,2022-03-01 16:08:22,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,
139767,B,2022-03-01 16:08:42,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,
139840,B,2022-03-01 16:09:01,41.87349,-87.80696,120,11040,11700,6.183922,Blu,to,O'Hare,False,ttpm_predslip_blue,


In [131]:
# We have multiple entries for some stations (the train might stay there for a minute)
ForestParkToHarlemForestPark = edf.query(f'RunNumber == 120 and CurrentStationId.isin([{BlueForestPark}, {BlueHarlemForestPark}])')
ForestParkToHarlemForestPark_dedup = ForestParkToHarlemForestPark[ForestParkToHarlemForestPark.CurrentStationId.diff() != 0]

# compute the difference
ForestParkToHarlemForestPark_dedup.Datetime.diff()

139466               NaT
139694   0 days 00:01:01
370768   3 days 04:02:00
370962   0 days 00:00:59
50632    1 days 00:35:20
50823    0 days 00:01:20
80659    0 days 03:33:20
80787    0 days 00:01:01
Name: Datetime, dtype: timedelta64[ns]

In [132]:
# the actual times are betweeen 31735 and 31720 so we want every other one, starting with 2
ForestParkToHarlemForestPark_dedup.Datetime.diff().iloc[1::2] # 2 is the start, empty is the end and 2 is the step size

139694   0 days 00:01:01
370962   0 days 00:00:59
50823    0 days 00:01:20
80787    0 days 00:01:01
Name: Datetime, dtype: timedelta64[ns]

### Prediction

In [133]:
import pandas as pd
pdf = pd.read_parquet('2022-03-preds-start.parquet')
print(pdf.query('RunNumber == 120 and Datetime < "2022-03-01 12:00:00"').to_string())

      Line            Datetime  CurrentStationId  RunNumber  StopOrder  ParentStop                StopName Prediction
204      B 2022-03-01 00:00:01             11602        120          0       40750  Harlem (O'Hare Branch)          3
205      B 2022-03-01 00:00:01             11602        120          1       40230              Cumberland          6
206      B 2022-03-01 00:00:01             11602        120          2       40820                Rosemont          9
692      B 2022-03-01 00:00:21             11604        120          0       40750  Harlem (O'Hare Branch)          2
693      B 2022-03-01 00:00:21             11604        120          1       40230              Cumberland          6
694      B 2022-03-01 00:00:21             11604        120          2       40820                Rosemont          8
1016     B 2022-03-01 00:00:41             11607        120          0       40750  Harlem (O'Hare Branch)          2
1017     B 2022-03-01 00:00:41             11607        

In [134]:
station_id_map.head(50)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,route_id,line,dir_code,geometry,index_right,Line,CurrentStationId,Lat,Lng,DirCode
0,30001,NaN,Austin-Blue,None,41.870851,-87.776812,0,40010,0,Blue,B,1,POINT (-87.77681 41.87085),3941,B,11080,41.87089,-87.77681,1
1,30012,NaN,Belmont-Blue,None,41.938132,-87.712359,0,40060,0,Blue,B,1,POINT (-87.71236 41.93813),1245,B,11520,41.93813,-87.71236,1
2,30014,NaN,Jackson-Blue,None,41.878183,-87.629296,0,40070,1,Blue,B,1,POINT (-87.62930 41.87818),2150,B,11300,41.87818,-87.62930,1
3,30034,NaN,Oak Park-Blue,None,41.872108,-87.791602,0,40180,0,Blue,B,1,POINT (-87.79160 41.87211),329,B,11060,41.87211,-87.79160,1
4,30042,NaN,Western-Forest Park,None,41.875478,-87.688436,0,40220,0,Blue,B,1,POINT (-87.68844 41.87548),7177,B,11160,41.87553,-87.68844,1
5,30044,NaN,Cumberland,None,41.984246,-87.838028,0,40230,1,Blue,B,1,POINT (-87.83803 41.98425),1232,B,11631,41.98429,-87.83803,1
6,30044,NaN,Cumberland,None,41.984246,-87.838028,0,40230,1,Blue,B,1,POINT (-87.83803 41.98425),1006,B,11622,41.98429,-87.83803,1
7,30044,NaN,Cumberland,None,41.984246,-87.838028,0,40230,1,Blue,B,1,POINT (-87.83803 41.98425),1310,B,11633,41.98429,-87.83803,1
8,30044,NaN,Cumberland,None,41.984246,-87.838028,0,40230,1,Blue,B,1,POINT (-87.83803 41.98425),1388,B,11640,41.98429,-87.83803,1
9,30048,NaN,Kedzie-Homan,None,41.874341,-87.706040,0,40250,1,Blue,B,1,POINT (-87.70604 41.87434),6552,B,11140,41.87442,-87.70605,1


In [135]:
# want a map from parent_station (aka ParentStop in the pdf) to CurrentStationId for Brown Line
# station_id_map.CurrentStationId // 1000 == 31 find the stations on the brown line (3) going in direction (1) (Kimball)
blue_entries = station_id_map[(station_id_map.line == 'B') & (station_id_map.CurrentStationId // 1000 == 11)].sort_values('CurrentStationId')[['stop_name', 'parent_station','CurrentStationId']]

In [136]:
blue_entries.query('stop_name == "Forest Park"')

,stop_name,parent_station,CurrentStationId
17,Forest Park,40390,11020


In [137]:
bluemap = blue_entries[['parent_station','CurrentStationId']].set_index('parent_station').CurrentStationId.to_dict()